In [1]:
import pandas as pd
import numpy as np
from haversine import haversine

In [6]:
data = pd.read_csv('./data/trade_data.csv')
data = data[['id', 'latitude', 'longitude']]
print(data.head())
print(len(data.values))

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


   id   latitude   longitude
0   0  37.663649  127.045058
1   1  37.481215  127.052733
2   2  37.481215  127.052733
3   3  37.476763  127.057210
4   4  37.476763  127.057210
917908


## 집 위치 정보 load

In [7]:
# houseFile = open('./data/house_loc.txt', 'r')
# houseFile = houseFile.read().split('\n')[:-1]
houseFile = data.values.tolist()

houseLoc = {}
for content in houseFile:
#     content = content.split(' ')
    houseLoc[str(int(content[0]))] = tuple(map(float, content[1:]))
print(len(houseLoc))
# houseLoc - {매물번호: (위도, 경도))}
# 위치 기준 정렬
# sortedHouseLoc = sorted(houseLoc.items(), key=(lambda x: x[1]))

917908


## 중복 좌표 제거

In [8]:
houseLL = sorted(list(set(houseLoc.values())))

print(len(houseLL))

6637


# 1. 지하철 정보 추출
## 1.1.지하철 정보 보충

In [9]:
# 2010년 이후로 추가된 역 더 추가
# stationListFile = open('station_list.txt', 'r', encoding='utf-8')
# stationListFile = stationListFile.read().split('\n')[:-1]
# s1 = set(stationListFile)
# print(len(s1))

stationLocFile = open('./data/station_loc.txt', 'r', encoding='utf-8')
stationLocFile = stationLocFile.read().split('\n')[:-1]
s2 = set([x.split(' ')[0] for x in stationLocFile])
print(len(s2))
# print(len(s1 - s2))
# with open('station_add.txt', 'w', encoding='utf-8') as f:
#     for x in (s1 - s2):
#         f.write(f'{x}\n')

476


## 1.2. 지하철 정보 -> dict

In [10]:
stationLoc = {}

for content in stationLocFile:
    content = content.split(' ')
    stationLoc[tuple(map(float, content[1:]))] = content[0]
    

# print(stationLoc) # {(위도, 경도): 역명}

# 위치 기준 정렬
# sortedStationLoc = sorted(stationLoc.items(), key=(lambda x: x[0]))
#print(sortedStationLoc)
#print(sortedStationLoc)

## 1.3. 좌표별 가장 가까운 점과의 거리 계산

In [11]:
shortest_dist = {} # {단지위치: (가장 가까운 역까지의 거리, 가장 가까운 역)}

for hLL in houseLL:
        d = []
        for sLL, sname in stationLoc.items():
            d.append((haversine(hLL, sLL, unit='m'), sname))
        d = sorted(d, key=lambda x: x[0])
        shortest_dist[hLL] = d[0]
#         print(f'{houseLoc[hLL]} {d[0][0]:.2f} {d[0][1]}')
print(len(shortest_dist))


6637


## 1.4. 매물과 좌표, 지하철 정보 match, File Out

In [12]:
print(len(houseLoc))
with open('./result/id_dist_station.txt', 'w') as f:
    for hcode, hLL in houseLoc.items():
        f.write(f'{hcode} {shortest_dist[hLL][0]:.2f} {shortest_dist[hLL][1]}\n')

917908


# 2. 병원 정보 추출
## 2.1. 병원 정보 -> dict

In [13]:
hospitalLoc = {}

hospitalLocFile = open('./data/hospital_loc.txt', 'r', encoding='utf-8')
hospitalLocFile = hospitalLocFile.read().split('\n')[:-1]

for content in hospitalLocFile:
    content = content.split(' ')
    hospitalLoc[tuple(map(float, content[1:]))] = content[0]
print(len(hospitalLoc))

53


## 2.2. 좌표별 가장 가까운 점과의 거리 계산

In [14]:
shortest_dist = {} # {단지 위치: (가장 가까운 병원까지의 거리, 가장 가까운 병원)}

for hLL in houseLL:
        d = []
        for sLL, sname in hospitalLoc.items():
            d.append((haversine(hLL, sLL, unit='m'), sname))
        d = sorted(d, key=lambda x: x[0])
        shortest_dist[hLL] = d[0]

print(len(shortest_dist))

6637


## 2.3. 매물과 좌표, 병원 정보 match, File Out

In [15]:
print(len(houseLoc))
with open('./result/id_dist_hospital.txt', 'w') as f:
    for hcode, hLL in houseLoc.items():
        f.write(f'{hcode} {shortest_dist[hLL][0]:.2f} {shortest_dist[hLL][1]}\n')

917908


# 3. 한강까지의 거리 구하기
## 1.1. cross 함수 정의

In [16]:
## 좌표계에서 두 점 사이의 거리를 구함
def cross_moonki(p1, p2, p):
    # 직선 위의 두 점
    b, a = p1
    d, c = p2
    # 직선에서 떨어진 점
    y1, x1 = p
    # t : 직선의 기울기
    t = (d - b) / (c - a)
    # x, y : 직교점
    x = (a * t * t - b * t + x1 + y1 * t)/(t * t + 1)
    y = t * x - t * a + b
    
    return (y, x)

## 1.2. 한강의 모양 정의
- 한강을 직선들의 연결로 정의한다

In [24]:
# File Input으로 수정
hangangPointFile = open('./data/hangang_point.txt', 'r', encoding='utf-8')
hangangPointFile = hangangPointFile.read().split('\n')[:-1]

hangang = [tuple(map(float, pointStr.split(' '))) for pointStr in hangangPointFile]
print(hangang)
# for pointStr in hangangPointFile:
#     point = tuple(map(float, pointStr.split(' ')))
#     hangang.append(point)


# hangang = [ (37.612300, 126.787429),
#             (37.554405, 126.883223),
#             (37.508452, 126.983883),
#             (37.538153, 127.025480),
#             (37.520164, 127.091548),
#             (37.567125, 127.127980),
#             (37.589338, 127.195365),
#             (37.553042, 127.232653)]

[(37.577416, 127.16131), (37.572416, 127.150281), (37.571327, 127.140625), (37.568674, 127.130625), (37.562313, 127.122428), (37.558299, 127.119339), (37.550712, 127.115905), (37.545064, 127.11333), (37.538904, 127.10891), (37.534063, 127.104506), (37.529605, 127.099915), (37.523648, 127.091074), (37.522219, 127.078714), (37.524874, 127.066312), (37.530455, 127.056184), (37.535152, 127.039919), (37.538283, 127.02816), (37.53573, 127.020478), (37.526814, 127.014213), (37.520251, 127.004651), (37.51569, 126.997613), (37.511435, 126.987442), (37.51072, 126.97534), (37.514941, 126.963624), (37.522668, 126.949462), (37.53036, 126.940235), (37.537608, 126.928133), (37.537983, 126.918735), (37.539616, 126.910323), (37.543938, 126.903671), (37.550028, 126.89247), (37.55833, 126.879424), (37.563875, 126.870541), (37.570338, 126.860241), (37.577141, 126.847495), (37.585065, 126.832518), (37.59207, 126.820201), (37.597001, 126.811875)]


## 1.3. 좌표별 한강까지의 거리 계산

In [25]:
shortest_dist = {} # {단지 위치: 가장 가까운 직선까지의 거리}

for hLL in houseLL:
        d = []
        for i in range(1, len(hangang)):
            npLL = cross_moonki(hangang[i-1], hangang[i], hLL)
            if hangang[i-1][0] > npLL[0] or hangang[i][0] < npLL[0]:
                continue
            d.append(haversine(hLL, npLL, unit='m'))
        for hangangLL in hangang:
            d.append(haversine(hLL, hangangLL, unit='m'))
        sorted(d)
        shortest_dist[hLL] = d[0]

print(sorted(shortest_dist.items(), key=lambda x: x[1]))

[((37.527354, 127.053394), 411.3476997148896), ((37.52813, 127.050251), 425.2076285439422), ((37.528007, 127.050603), 427.4556217688155), ((37.527524, 127.052159), 430.9998081371176), ((37.528144, 127.049903), 434.3149583555701), ((37.528224, 127.049582), 435.6519693583501), ((37.526856, 127.054215), 436.0065189927465), ((37.528425, 127.048844), 436.9279896429653), ((37.52867, 127.047845), 441.5071861740565), ((37.528757, 127.047471), 443.71868414741556), ((37.528429, 127.048176), 456.81829828497405), ((37.528858, 127.04658700000002), 459.9633424831017), ((37.527753, 127.050368), 461.34558489871773), ((37.536371, 127.050915), 462.70016372810517), ((37.537739, 127.046191), 463.1004033122843), ((37.52668, 127.05395), 465.40932833578756), ((37.52646, 127.054288), 469.1236149753299), ((37.528618, 127.047076), 470.36494968246365), ((37.528287, 127.048213), 470.6448806884161), ((37.528372, 127.047792), 474.4960643818039), ((37.529975, 127.042172), 476.595227218995), ((37.528686, 127.046533),

## 1.4. 매물과 좌표, 한강까지의 거리 File Out

In [26]:
print(len(houseLoc))
with open('./result/id_dist_hangang-center.txt', 'w') as f:
    for hcode, hLL in houseLoc.items():
        f.write(f'{hcode} {shortest_dist[hLL]:.2f}\n')

917908


## 번외: hangang-top, hangang-bottom 중 짧은 거리 적용

In [27]:
# File 불러오기
hangangDistTopFile = open('./result/id_dist_hangang-top.txt', 'r', encoding='utf-8')
hangangDistBottomFile = open('./result/id_dist_hangang-bottom.txt', 'r', encoding='utf-8')

hangangDistTopFile = hangangDistTopFile.read().split('\n')[:-1]
hangangDistBottomFile = hangangDistBottomFile.read().split('\n')[:-1]

shortest = {}
# 1st: hangangDistTop
for info in hangangDistTopFile:
    info = info.split(' ')
    shortest[info[0]] = float(info[1])

# 2nd: hangangDistBottomFile
for info in hangangDistBottomFile:
    info = info.split(' ')
    if info[0] in shortest:
        shortest[info[0]] = min(shortest[info[0]], float(info[1]))
    else:
        shortest[info[0]] = float(info[1])

# 출력 테스트
# print(sorted(shortest.items(), key=lambda x: x[1]))
# 너무 길어서 jupyter에서 출력 안됨

In [28]:
# 최종 최단거리 기록
with open('./result/id_dist_hangang-top+bottom.txt', 'w') as f:
    for hcode, dist in shortest.items():
        f.write(f'{hcode} {dist:.2f}\n')

In [50]:
# 단지번호 입력하면 좌표 찍는 테스트용 코드
num = '1341698'
houseLoc[num]

(37.53553400000001, 127.069177)